# ACDC : l'Autre Carré D'à Côté

alias "la zone à JPD"

alias "le plateau de Cournols-Olloix"

(pour faire du DS évidemment)

In [ ]:
import sys
import math
import re
import datetime as dt
import pandas as pd
import numpy as np

# Cartographie des points ACDC 2019

1. charger KML donnant les limites de la zone
    * lecture KML => polygone zone en coordonnées sphériques
    * conversion / projection en coordonnées métriques : UTM 31
2. générer les points sur la grille et dans les limites de la zone, + certaine distance pour élargir ?
    * grille UTM 31 + qq 100m (qq entier)
    * 1er jet rectancle circonscrit à la zone (xMin, yMin, xMax, yMax) + marge
    * élimination des points hors du polygone de la zone ciblée + marge
3. exporter en KML

## 1a) Charger KML donnant les limites de la zone

In [ ]:
from lxml import etree

In [ ]:
kmlNameSpaces = \
  { 'gx' : 'http://www.google.com/kml/ext/2.2',
    'kml' : 'http://www.opengis.net/kml/2.2',
    'atom' : 'http://www.w3.org/2005/Atom' }

In [ ]:
kmlRoot = etree.ElementTree().parse('ACDC2019-limites-zone.kml')

In [ ]:
# On suppose que c'est le 1er polygône
plMark = kmlRoot.find('kml:Document/kml:Document/kml:Placemark', namespaces=kmlNameSpaces)
zonePoly = plMark.find('kml:Polygon/kml:outerBoundaryIs/kml:LinearRing/kml:coordinates',
                         namespaces=kmlNameSpaces).text.strip()
dfZonePoly = pd.DataFrame(data=[[float(v) for v in point.split(',')] for point in zonePoly.split(' ')],
                          columns=['long', 'lat', 'alt'])
dfZonePoly.head()

## 1b) Conversion coords polygône en métriques

In [ ]:
import pyproj

In [ ]:
# Projection coordonnées sphériques (degrés)=> coordonnées planes (système cible au choix)
KProjWgs84 = pyproj.Proj(init='epsg:4326') # WGS 84 : long, lat en degrés

KProjUtm31  = pyproj.Proj(init='epsg:32631') # WGS 84 - UTM 31N : long, lat en m
KProjCc46   = pyproj.Proj(init='epsg:3946')  # RGF 93 - CC46    : long, lat en m
KProjLamb93 = pyproj.Proj(init='epsg:2154')  # RGF 93 - Lambert : long, lat en m

def geoProjeter(sCoords, srcProj, tgtProj): # sCoords : [0]=x=long, [1]=y=lat
    return pd.Series(pyproj.transform(srcProj, tgtProj, sCoords[0], sCoords[1]))

In [ ]:
# Attention : Les colonnes sources (x,y) et (x_observateur,y_observateur)
#             sont bizarement des couples (lat, long), et pas l'inverse.
KInfValues = [np.inf, -np.inf]
dfZonePoly[['xUtm', 'yUtm']] = \
  dfZonePoly[['long', 'lat']].apply(geoProjeter, srcProj=KProjWgs84, tgtProj=KProjUtm31, axis='columns')
dfZonePoly[['xUtm', 'yUtm']] = dfZonePoly[['xUtm', 'yUtm']].replace(KInfValues, np.nan) #, inplace=True)

In [ ]:
dfZonePoly.head()

In [ ]:
# Rectangle circonscrit à la zone
xUtmMin, yUtmMin, xUtmMax, yUtmMax = \
    dfZonePoly.xUtm.min(), dfZonePoly.yUtm.min(), dfZonePoly.xUtm.max(), dfZonePoly.yUtm.max()
xUtmMin, yUtmMin, xUtmMax, yUtmMax

## 2a) Détermination / choix de la taille des cellules de la grille

In [ ]:
from shapely import geometry

In [ ]:
# Le polygone de la zone (et sa surface en ha).
geoZonePoly = geometry.Polygon(shell=[(x, y) for x, y in dfZonePoly[['xUtm', 'yUtm']].itertuples(index=False)])

geoZonePoly.area / 10000

In [ ]:
# Nombre de points approximatif à répartir sur la zone.
nPoints = 150
txCouver = 60 # % ; rapide calcul après avoir dit : 100 points sur 2000 ha de milieux cibles (on vire les forêts)

In [ ]:
# Surface couverte par 1 point
surfPoint = geoZonePoly.area / nPoints
surfPoint / 10000, 'ha'

In [ ]:
# Soit un cercle de diamètre ...
deltaXYPoints = 2 * math.sqrt(surfPoint) * 100 / txCouver / math.pi
deltaXYPoints

In [ ]:
# Bon, on prend plutôt ...
deltaXYPoints = 500 #400

## 2b) Premier jet de points dans rectangle circonscrit + marge d'1 point

In [ ]:
# Rectangle circonscrit + marge d'1 point
xUtmMinR = xUtmMin - deltaXYPoints / 2
xUtmMaxR = xUtmMax + deltaXYPoints / 2
yUtmMinR = yUtmMin - deltaXYPoints / 2
yUtmMaxR = yUtmMax + deltaXYPoints / 2

In [ ]:
# Alignement sur une grille UTM à N m, avec ajustement par décalage en X, Y si besoin
uniteAlign = 100 # m
offsetX = 0
offsetY = 0

xUtmMinR = uniteAlign * math.floor(xUtmMinR / uniteAlign) + offsetX
yUtmMinR = uniteAlign * math.floor(yUtmMinR / uniteAlign) + offsetY
xUtmMaxR = uniteAlign * math.ceil(xUtmMaxR / uniteAlign) + offsetX
yUtmMaxR = uniteAlign * math.ceil(yUtmMaxR / uniteAlign) + offsetY

xUtmMinR, yUtmMinR, xUtmMaxR, yUtmMaxR

In [ ]:
dfPoints = pd.DataFrame(data=[dict(xUtm=x, yUtm=y) \
                              for y in np.arange(yUtmMaxR, yUtmMinR - deltaXYPoints, -deltaXYPoints) \
                              for x in np.arange(xUtmMinR, xUtmMaxR + deltaXYPoints, deltaXYPoints)])
dfPoints['numero'] = range(1, len(dfPoints)+1)
dfPoints[['long', 'lat']] = \
  dfPoints[['xUtm', 'yUtm']].apply(geoProjeter, srcProj=KProjUtm31, tgtProj=KProjWgs84, axis='columns')
dfPoints.set_index('numero', inplace=True)
dfPoints.head()

## 2c) Supprimer les points hors zone + marge

In [ ]:
# Marge en distance au delà de l'appartenance au polygône de la zone.
marginDist = deltaXYPoints * 1.0 #* 0.25

In [ ]:
def pointAroundZone(sXYPoint):
    point = geometry.Point(sXYPoint)
    #return geoZonePoly.contains(point) or point.distance(geoZonePoly) < marginDist
    return point.distance(geoZonePoly) < marginDist # Pas besoin de tester l'appartenance, distance() le fait.
dfPoints['aroundZoneExt'] = \
    dfPoints[['xUtm', 'yUtm']].apply(pointAroundZone, axis='columns')
len(dfPoints), len(dfPoints[dfPoints.aroundZoneExt])

In [ ]:
dfSelPoints = dfPoints[dfPoints.aroundZoneExt]
dfSelPoints.head()

## 3a) Cartographie des points obtenus

In [ ]:
import folium

In [ ]:
tiles, attr = 'http://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'osm'# OK
#tiles, attr = 'https://{s}.tile.thunderforest.com/outdoors/{z}/{x}/{y}.png', 'thunderforest' # OK
#tiles, attr = 'http://{s}.tile.openstreetmap.fr/fradm/{z}/{x}/{y}.png', 'osm fr' # marche pô
#tiles, attr = 'https://{s}.tile.openstreetmap.fr/qa/{zoom}/{x}/{y}.png', 'osm fr' # marche pô
mp = folium.Map(tiles=tiles, attr=attr)

poly = folium.PolyLine(locations=[(lat, long) for long, lat in dfZonePoly[['long', 'lat']].itertuples(index=False)],
                       color='red', opacity=0.8, popup='Zone ACDC Cournols-Olloix JPD')
poly.add_to(mp)
for indPt, sPt in dfSelPoints.iterrows():
    mrk = folium.Marker(location=(sPt.lat, sPt.long), 
                        popup=folium.Popup('{} : lat={:.1f}, long={:.1f}'.format(indPt, sPt.lat, sPt.long)),
                        icon=folium.Icon(color='green', icon_color='black'))
    mrk.add_to(mp)
    
mp.fit_bounds(mp.get_bounds())
mp

## 3b) Export KML

In [ ]:
dfSelPoints.head()

In [ ]:
import simplekml as skml # Simple KML generation API

In [ ]:
kml = skml.Kml(name='Points ACDC 2019 (dist={:.0f}m, marge={:.0f}m, n={})' \
               .format(deltaXYPoints, marginDist, len(dfSelPoints)))

In [ ]:
labelStyle = skml.LabelStyle(color=skml.Color.red, scale=1)
iconStyle = skml.IconStyle(icon=skml.Icon(href='http://maps.google.com/mapfiles/kml/shapes/placemark_circle.png'))
ptStyle = skml.Style(labelstyle=labelStyle, iconstyle=iconStyle)

lineStyle = skml.LineStyle(color=skml.Color.red, width=3)

In [ ]:
ls = kml.newlinestring(name='Zone ACDC Cournols-Olloix JPD', extrude=1,
                       coords=dfZonePoly[['long', 'lat', 'alt']].values)
                       #coords=[(long, lat, alt) for long, lat, alt in dfZonePoly[['long', 'lat', 'alt']].itertuples(index=False)])
ls.linestyle = lineStyle

for idx, sPt in dfSelPoints.iterrows():
    pt = kml.newpoint(name=str(idx), coords=[(sPt.long, sPt.lat, 0)], extrude=1)
    pt.style = ptStyle
    pt.description = 'lat={:.1f}, long={:.1f}, alt={:.0f}'.format(sPt.long, sPt.lat, 0)

In [ ]:
tgtKmlFileName = \
  'ACDC2019-{}points-et-limites-zone-d{:.0f}-m{:.0f}.kml'.format(len(dfSelPoints), deltaXYPoints, marginDist)
kml.save(tgtKmlFileName)

## 3c) Export Excel

In [ ]:
tgtXlsxFileName = \
  'ACDC2019-{}points-et-limites-zone-d{:.0f}-m{:.0f}.xlsx'.format(len(dfSelPoints), deltaXYPoints, marginDist)

dfSelPoints[['xUtm', 'yUtm', 'long', 'lat']].reset_index().to_excel(tgtXlsxFileName, index=False)